In [ ]:

%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

path_to_add = Path("/home/ronkanto/data/machine_learning/")
if str(path_to_add) not in sys.path:
    sys.path.append(str(path_to_add))

import numpy as np
import pandas as pd
import torch
import traceback


from src.model import PoolingLayerEnum, MatrixEncoderType, MatrixEncoderActivationType
from src.experiment import SingleParameterExperimentCreation
from src.data_processing.data_loading import SamplePath


RANDOM_STATE = 71070


DEVICE = "cuda:3"

In [ ]:
default_model_parameters = {
    "convolution_channels": [16] * 3,
    "num_graph_layers": 4,
    "random_state": RANDOM_STATE,
    "use_batchnorm": True,
    "dropout_probability": 0.5,
    "gnn_pooling_layer": PoolingLayerEnum.sum_pool,
    "encoder_type": MatrixEncoderType("full"),
    "gnn_activation": "GELU",
    "activation_type": MatrixEncoderActivationType(
        activation_type="lrelu", activation_params=dict(lrelu_slope=0.01)
    ),
    "gnn_network_type": "graphsage"
}

default_experiment_parameters = {
    "batch_size": 512,
    "num_folds": 5,
    "random_state": RANDOM_STATE,
    "sample_path": SamplePath.old_smoothing,
    "num_workers": 8,
    "synthesize_negative_samples": False,
    "max_matrix_noise": 0,
    "max_positive_upsampling_per_pathway": 1,
    "max_matrix_nmae": 0.2,
}

experiment_sets = [
    ("main_model", ["graphsage"], True)
]


tb = None
exception = None



try:

    for experiment_parameter_name, experiment_parameter_values, is_model_parameter in experiment_sets:
        print(experiment_parameter_name, experiment_parameter_values)
        experiment = SingleParameterExperimentCreation(
            default_model_parameters=default_model_parameters,
            default_experiment_parameters=default_experiment_parameters,
            changing_parameter_name=experiment_parameter_name,
            changing_parameter_values=experiment_parameter_values,
            is_model_parameter=is_model_parameter,
            experiment_name_prefix="gat_",
            results_directory=Path(f"../results/model_param_runs/epochs_200/{experiment_parameter_name}/"),
        ).create_experiment()
        experiment.run_and_save_results(num_epochs=200, with_tqdm=True, device=DEVICE)
    

except Exception as e:
    exception = e
    tb = traceback.format_exc()
    raise

